- load the saved Pipeline Model as `pipelineModel`

as part of the streaming processing:
- load streaming data from directory 
- use the pipeline model to transform the streaming dataframe.
- drop the unwanted intermediarely columns in the output
- output to a `memory` sink (`scored_tweets`) using the append mode.

In [0]:
%%bash

rm -f tweets.zip
rm -rf tweets
wget -nv http://idsdl.csom.umn.edu/c/share/msba6330/tweets.zip
unzip tweets.zip -d tweets

Archive:  tweets.zip
  inflating: tweets/20221105-123837.txt  
  inflating: tweets/20221105-123835.txt  
  inflating: tweets/20221105-123833.txt  
  inflating: tweets/20221105-123831.txt  
  inflating: tweets/20221105-123828.txt  
  inflating: tweets/20221105-123825.txt  
  inflating: tweets/20221105-123822.txt  
  inflating: tweets/20221105-123821.txt  
  inflating: tweets/20221105-123819.txt  
  inflating: tweets/20221105-123816.txt  
  inflating: tweets/20221105-123814.txt  
  inflating: tweets/20221105-123813.txt  
  inflating: tweets/20221105-123811.txt  
  inflating: tweets/20221105-123809.txt  
  inflating: tweets/20221105-123806.txt  
  inflating: tweets/20221105-123804.txt  
  inflating: tweets/20221105-123803.txt  
  inflating: tweets/20221105-123801.txt  
  inflating: tweets/20221105-123759.txt  
  inflating: tweets/20221105-123756.txt  
  inflating: tweets/20221105-123754.txt  
  inflating: tweets/20221105-123752.txt  
  inflating: tweets/20221105-123749.txt  
  inflating: 

In [0]:
from pyspark.ml import PipelineModel
modelPath = "/FileStore/twitter_nbpipeline/"
inputPath = "file:/databricks/driver/tweets"

pipelineModel  = PipelineModel.load(modelPath)
streamingInputDF = spark.readStream.option("maxFilesPerTrigger",1).schema("time timestamp, text string").json(inputPath)

scored_tweets =  pipelineModel.transform(streamingInputDF)

query = scored_tweets.drop("rowPrediction","Probability","features","words","words_filtered").writeStream.trigger(processingTime="2 seconds").format("memory").queryName("scored_tweets").outputMode("append").start()


In [0]:
#view stream result
%sql
select count(*) from scored_tweets;

In [0]:
%sql
select sum(if(prediction=1,1,0)) as positive, sum(if(prediction=0,1,0)) as negative, window(time,"30 seconds") from scored_tweets 
group by window(time,"30 seconds")


positive,negative,window
1,0,"List(2022-11-05T17:36:00.000+0000, 2022-11-05T17:36:30.000+0000)"
59,73,"List(2022-11-05T17:36:30.000+0000, 2022-11-05T17:37:00.000+0000)"
18,19,"List(2022-11-05T17:37:00.000+0000, 2022-11-05T17:37:30.000+0000)"
